<a href="https://colab.research.google.com/github/mgalbis/movie-recommender-system/blob/main/PRA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Práctica Sistema de recomendación

## Introducción

En la siguiente práctica se van a desarrollar una serie de sistemas de recomendación:

1. Filtrado colaborativo (item-based)
2. Filtrado colaborativo (user-based)
3. Filtrado basado en contenido (item-based)

**Autora:** María Galbis Calomarde

El siguiente bloque descarga el zip del dataset (Si este bloque fallara, subir manualmente el dataset, e ir directamente al siguiente bloque)

In [21]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

#url = "https://drive.google.com/uc?id=1toh8_1NaYfIQ9f2Et95JjVw-7UrViBJ7&export=download"
#html=!curl --cookie-jar 'cookie' -sq "{url}"

#print(html)
#soup = BeautifulSoup(''.join(html))
#url = 'https://drive.google.com' + soup.find(id='uc-download-link')['href']
#print(url)

url = "https://drive.google.com/uc?id=1s-dBE-TOOMuebQL3DaY-sqHooL4iUBFw&export=download&confirm=t"

!curl -L -o "movie_dataset.zip" "{url}"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 58.6M  100 58.6M    0     0  85.3M      0 --:--:-- --:--:-- --:--:--  222M


En el siguiente bloque se extraen los datos en dataframes

In [22]:
from zipfile import ZipFile
import pandas as pd

with ZipFile('movie_dataset.zip', 'r') as zfile:
  with zfile.open('movielens_movie_ratings.csv') as dr:
    ratings = pd.read_csv(dr, delimiter='|')

  with zfile.open('movielens_movie_titles.csv') as dr:
    titles = pd.read_csv(dr, delimiter='|')

  with zfile.open('tmdb_movies_detail.csv') as dr:
    details = pd.read_csv(dr, delimiter='|')

In [23]:
ratings

,userId,movieId,rating
0,1,8844,3.5
1,1,902,3.5
2,1,63,3.5
3,1,807,3.5
4,1,629,3.5
...,...,...,...
19987676,138493,14160,4.5
19987677,138493,8373,4.5
19987678,138493,8355,3.0
19987679,138493,17654,5.0


In [24]:
titles

,movieId,title
0,862,Toy Story (1995)
1,8844,Jumanji (1995)
2,15602,Grumpier Old Men (1995)
3,31357,Waiting to Exhale (1995)
4,11862,Father of the Bride Part II (1995)
...,...,...
27021,4436,Kein Bund für's Leben (2007)
27022,9274,"Feuer, Eis & Dosenbier (2002)"
27023,285213,The Pirates (2014)
27024,32099,Rentun Ruusu (2001)


In [25]:
details

,movieId,genres,title,original_language,overview,popularity,release_date,runtime,vote_average,vote_count
0,19995,"['Action', 'Adventure', 'Fantasy', 'Science Fi...",Avatar,en,"In the 22nd century, a paraplegic Marine is di...",150.437577,2009-12-10,162.0,7.2,11800
1,285,"['Adventure', 'Fantasy', 'Action']",Pirates of the Caribbean: At World's End,en,"Captain Barbossa, long believed to be dead, ha...",139.082615,2007-05-19,169.0,6.9,4500
2,206647,"['Action', 'Adventure', 'Crime']",Spectre,en,A cryptic message from Bond’s past sends him o...,107.376788,2015-10-26,148.0,6.3,4466
3,49026,"['Action', 'Crime', 'Drama', 'Thriller']",The Dark Knight Rises,en,Following the death of District Attorney Harve...,112.312950,2012-07-16,165.0,7.6,9106
4,49529,"['Action', 'Adventure', 'Science Fiction']",John Carter,en,"John Carter is a war-weary, former military ca...",43.926995,2012-03-07,132.0,6.1,2124
...,...,...,...,...,...,...,...,...,...,...
4798,9367,"['Action', 'Crime', 'Thriller']",El Mariachi,es,El Mariachi just wants to play his guitar and ...,14.269792,1992-09-04,81.0,6.6,238
4799,72766,"['Comedy', 'Romance']",Newlyweds,en,A newlywed couple's honeymoon is upended by th...,0.642552,2011-12-26,85.0,5.9,5
4800,231617,"['Comedy', 'Drama', 'Romance', 'TV Movie']","Signed, Sealed, Delivered",en,"""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,2013-10-13,120.0,7.0,6
4801,126186,[],Shanghai Calling,en,When ambitious New York attorney Sam is sent t...,0.857008,2012-05-03,98.0,5.7,7


## 1. Filtrado colaborativo (item-based)
Este tipo de filtrado se va a realizar mediante un algoritmo basado en el modelo KNearestNeighbors

In [26]:
MOVIE_ID = 10530    # Película de prueba
n_commends = 5      # Nº de recomendaciones


from scipy import sparse
from sklearn.neighbors import NearestNeighbors
import numpy as np

# Pivotamos los datos en una matriz dispersa para tratar los datos sin provocar 
# desbordamientos por memoria
ratings_mtx = sparse.csr_matrix((ratings.rating, (ratings.movieId, ratings.userId)))

# Entrenamos el algoritmo con todo el set de datos
knn = NearestNeighbors(metric='cosine')
knn.fit(ratings_mtx)

# Obtenemos los n_commends + 1 items más cercanos según rating
# (el primero será el mismo item que hemos pasado, por eso añadimos uno más)
[distances], [indices] = knn.kneighbors(
    ratings_mtx[MOVIE_ID], n_neighbors=n_commends + 1)

# Recorremos las distancias y los índices de las películas recomendadas
# Se pinta el id y la distancia, seguido del título de la película
for i in range(0, len(distances.flatten())):
  if i == 0:
    title = titles.loc[titles['movieId'] == MOVIE_ID, 'title'].values[0]
    print('Recomendaciones para {0}:'.format(title))
  else:
    title = titles.loc[titles['movieId'] == indices[i], 'title'].values[0]
    print('{0}: [{1:>8}, {2:.4f}] - {3}'.format(i, indices[i], distances[i], title))


Recomendaciones para Pocahontas (1995):
1: [   10020, 0.5531] - Beauty and the Beast (1991)
2: [    8587, 0.5642] - Lion King, The (1994)
3: [    8839, 0.5647] - Casper (1995)
4: [     812, 0.5769] - Aladdin (1992)
5: [     408, 0.5980] - Snow White and the Seven Dwarfs (1937)


## 2. Filtrado colaborativo (user-based)
Este tipo de filtrado también se va a realizar mediante un algoritmo basado en el modelo KNearestNeighbors, pero esta vez, al obtener los usuarios similares, se hará una selección de las películas vistas por cada uno de ellos, y las más repetidas serán las recomendadas.

In [37]:
USER_ID = 326      # Usuario de prueba
n_commends = 5   # Nº de recomendaciones

from scipy import sparse
from sklearn.neighbors import NearestNeighbors
import numpy as np
import operator

# Pivotamos los datos en una matriz dispersa para tratar los datos sin provocar 
# desbordamientos por memoria
ratings_mtx = sparse.csr_matrix((ratings.rating, (ratings.userId, ratings.movieId)))

# Entrenamos el algoritmo con todo el set de datos
knn = NearestNeighbors(metric='cosine')
knn.fit(ratings_mtx)

# Obtenemos los n_commends + 1 usuarios más cercanos según rating
# (el primero será el mismo item que hemos pasado, por eso añadimos uno más)
[distances], [indices] = knn.kneighbors(
    ratings_mtx[USER_ID], n_neighbors=n_commends + 1)

# Creamos un diccionario para almacenar las películas no vistas por el usuario
# como key y el número de veces que se repite como value
rec_movies = dict()
# las películas vistas por el usuario actual
us_movies = np.array(ratings.loc[ratings['userId'] == USER_ID]['movieId'].values)
for i in range(0, len(distances.flatten())):
  if i > 0: # i == 0 es la película actual
    # las películas de la película iterada
    cu_movies = np.array(ratings.loc[ratings['userId'] == indices[i]]['movieId'].values)
    # le eliminamos las películas ya vistas por el usuario
    cu_movies = cu_movies[~np.isin(cu_movies, us_movies)]
    # las almacenamos en el diccionario 
    for m in cu_movies:
      rec_movies[m] = rec_movies[m] + 1 if m in rec_movies else 1

# ordenamos el diccionario según el valor de forma decreciente
rec_movies = dict(sorted(rec_movies.items(), key=operator.itemgetter(1), reverse=True))


# Pintamos las películas vistas por el usuario
for i, m in enumerate(us_movies):
  if i == 0:
    print('Películas vistas por el usuario {0}:'.format(USER_ID))

  title = titles.loc[titles['movieId'] == m].values[0]
  print('{0:>2}: [{1:>8}] - {2}'.format(i+1, title[0], title[1]))

print('')

# Recorremos las películas recomendadas
# Se pinta el id y el número de veces que aparece entre los usuarios cercanos,
# seguido del título de la película
for i, m in enumerate(rec_movies):
  if i == 0:
    print('Recomendaciones para el usuario {0}:'.format(USER_ID))
  
  title = titles.loc[titles['movieId'] == m].values[0]
  print('{0}: [{1:>8}, {2:>2}] - {3}'.format(i+1, title[0], rec_movies[m], title[1]))
  if i == n_commends-1: break


Películas vistas por el usuario 326:
 1: [    8844] - Jumanji (1995)
 2: [    4584] - Sense and Sensibility (1995)
 3: [   16420] - Othello (1995)
 4: [     902] - City of Lost Children, The (Cité des enfants perdus, La) (1995)
 5: [   11010] - Postman, The (Postino, Il) (1994)
 6: [    9208] - Broken Arrow (1996)
 7: [   33542] - Rumble in the Bronx (Hont faan kui) (1995)
 8: [     628] - Interview with the Vampire: The Vampire Chronicles (1994)
 9: [      11] - Star Wars: Episode IV - A New Hope (1977)
10: [     101] - Léon: The Professional (a.k.a. The Professional) (Léon) (1994)
11: [   14334] - Secret of Roan Inish, The (1994)
12: [     193] - Star Trek: Generations (1994)
13: [    2759] - Adventures of Priscilla, Queen of the Desert, The (1994)
14: [     713] - Piano, The (1993)
15: [   11827] - Heavy Metal (1981)
16: [     954] - Mission: Impossible (1996)
17: [     602] - Independence Day (a.k.a. ID4) (1996)
18: [    3595] - Ransom (1996)
19: [    3573] - Emma (1996)
20: [     

## 3. Filtrado basado en contenido (item-based)

Este tipo de filtrado también se realiza mediante un algoritmo basado en el model KNearestNeighbors. Previamente, se ha tratado los datos para separar los géneros en diferentes columnas y marcarlas como [0,1], y se ha aplicado ranging.



In [39]:
MOVIE_ID = 10530    # Película de prueba
n_commends = 5      # Nº de recomendaciones

from scipy import sparse
from sklearn.neighbors import NearestNeighbors
import numpy as np

details_new = details.copy()

# Eliminamos las columnas que no queremos usar en el modelo: la duración de la película
# el título, el lenguaje original, la descripción, la fecha, y la media y el número de votos
# (estas dos últimas porque la columna popularity está calculada a partir de estas)
to_drop = ['title', 'overview', 'release_date', 'vote_average', 'vote_count']
details_new = details_new.drop(to_drop, axis=1).set_index("movieId")
details_new['genres'] = details_new['genres'].apply(lambda x: eval(x))

languages = list(set(details['original_language']))
details_new['original_language'] = details_new['original_language'].apply(lambda x: languages.index(x))

# creamos una columna por cada género y lo rellenamos con 1 o 0 según si pertenece
# a esa categoria o no
for i, row in details_new.iterrows():
  for g in row['genres']:
    if g not in details_new:
      details_new.insert(0, g, 0, allow_duplicates=False)
    details_new.at[i, g] = 1
    
details_new = details_new.drop('genres', axis=1)
details_new = details_new.fillna(0)


# ranging
for c in details_new.columns:
  max = details_new[c].max()
  min = details_new[c].min()

  details_new[c] = (details_new[c] - min) / (max - min)

knn = NearestNeighbors(metric='cosine')
knn.fit(details_new.values)

# Obtenemos los n_commends + 1 items más cercanos según rating
# (el primero será el mismo item que hemos pasado, por eso añadimos uno más)
[distances], [indices] = knn.kneighbors(
    [details_new.loc[MOVIE_ID].values], n_neighbors=n_commends + 1)

# Recorremos las distancias y los índices de las películas recomendadas
# Se pinta el id y la distancia, seguido del título de la película
for i in range(0, len(distances.flatten())):
  if i == 0:
    title = details.loc[details['movieId'] == MOVIE_ID, 'title'].values[0]
    print('Recomendaciones para {0}:'.format(title))
  else:
    movieId = details.loc[indices[i], 'movieId']
    title = details.loc[indices[i], 'title']
    print('{0}: [{1:>8}, {2:.4f}] - {3}'.format(i, movieId, distances[i], title))


Recomendaciones para Pocahontas:
1: [    9837, 0.1040] - The Prince of Egypt
2: [    7484, 0.1319] - Open Season
3: [   10545, 0.1319] - The Hunchback of Notre Dame
4: [   10674, 0.1319] - Mulan
5: [    3170, 0.1320] - Bambi
